&copy; Young Data Scientists League, 2022-2023

# [WIP] Escape Room: Predicting Passcodes

<div align="center"> 
<figure class="image">
  <img src="https://raw.githubusercontent.com/headfirstai/backprop_game/fc0564daaf28266192fa6b96b6cb9499038e5368/assets/unsplash_combo_lock.jpg", width = "60%">
  <!-- <figcaption>Source: https://www.census.gov/popclock/</figcaption> -->
</figure>
</div>


You and your team are trying to escape from a room that is secured by a special "smart lock" that is designed to be challenging to crack. Instead of having a single passcode like most locks do, this lock changes its passcode over time! To break this lock, three correct guesses in a row are required.

The lock displays a random sequence of numbers that changes every minute. Based on this sequence, it sets a new passcode. When this happens, your team has one minute to guess the passcode before it changes again.

- If the correct guess is entered, the lock will mark a tally (&#10004;) and display a new sequence. Three tallies will open the lock.

- If an incorrect guess is entered, the tally will reset and the lock will show what the correct passcode was. One minute later, it will display a brand new sequence and your team is allowed to guess again.

To escape from this room, you must learn how the lock generates its passcodes.

## Clues
Sounds tricky, right? Fortunately, your TA is an AI expert who studied this lock and noticed some patterns about how it works:

- The length of the display sequence never changes.
- The passcode is always an integer (for example, 5, or -17).
- Guesses that are close to the passcode (within a margin of +/- 3) are also counted as valid.

Your TA also noticed that some numbers in the sequence seem to be more important than others, and suspects that the lock is using a simple "linear combination" to set the passcodes:

1. It starts by creating a set of multipliers: random numbers that range from 1 to 9. There is one multiplier for each position in the sequence. These multipliers never change and remain the same even as the sequence changes.
2. For every new sequence, it multiplies each number in the sequence by its corresponding multiplier.
3. Finally, the lock sums up all the products to generate the passcode. It also adds a random small number (between -3 and 3) to make it harder to reverse engineer.

### Example

Suppose the lock always displays a sequence of length two. It starts by creating two multipliers as random numbers between 1 and 9. Let's say it chose 4 and 5. These multipliers are hidden and don't change for the duration of the game.

Now, if the lock displays a sequence of (1, 2), the passcode is calculated by first multiplying the sequence with the multipliers (4 * 1 + 5 * 2 = 4 + 10 = 14), and finally adding a small random number (14 + 1 = 15). Then, any guess from 12 to 18 will be accepted.

After a minute elapses, the lock will generate a new sequence, say (-1, 3). The passcode is calculated with the same procedure on the new sequence, first by multiplying: (4 * -1 + 5 * 3 = -4 + 15 = 11), and adding a small random number to the sum: (11 + 2 = 13). Then, any guess from 10 to 16 will be accepted.

Your TA suggests that estimating the multipliers is the key to guessing the passcodes. To simplify the math, your TA has created a calculator that produces guesses based on the procedure above.

**Ready to try it out? Run the cell below to get started.**

In [ ]:
## Play Game (Team or Individual)
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact, Layout
from IPython.display import display, HTML
from matplotlib import animation, rc
import matplotlib.pyplot as pyplot
import random
 
## Terminology
# n: sequence length (i.e. difficulty)
# x: input feature (i.e. number in sequence)
# SIGMA: parameter for generating varying passcodes (additive Gaussian noise)
 
## Setup: Constants
MIN_N = 1
MAX_N = 12
MIN_MULTIPLIER = 1
MAX_MULTIPLIER = 9
MIN_X = -2
MAX_X = 5
 
ALLOW_AI = False
IS_REPLAY = False
USE_STUDENT_AI = False
 
## Setup: Global Variables
global n
global multipliers
global SIGMA
global guess_leeway
 
global attempt_number
global guessing_enabled
global correct_tally
 
global multiplier_guesses
global display_sequence
global noise_term
 
global ai_multiplier_guesses
 
## Setup: Helper Functions
def generate_multipliers():
  global n, multipliers, MIN_MULTIPLIER, MAX_MULTIPLIER
  multipliers = [
    random.randint(MIN_MULTIPLIER, MAX_MULTIPLIER)
    for i in range(n)
  ]
 
def attempt_header_html():
  global attempt_number
  return f"<h2>Lock Attempt #{attempt_number}</h2>"
 
def cell_text(value, bold=False, color="black"):
  elem = "strong" if bold else "span"
  return f'<{elem} style="color:{color};">{str(value)}</{elem}>'
 
def generate_sequence():
  global n, display_sequence, MIN_X, MAX_X
  global IS_REPLAY, attempt_number
 
  if IS_REPLAY and attempt_number <= len(historical_sequences):
    display_sequence = historical_sequences[attempt_number - 1]
  else:
    while True:
      display_sequence = [
        random.randint(MIN_X, MAX_X)
        for i in range(n)
      ]
 
      if not min(display_sequence) == 0 or not max(display_sequence) == 0:
        break
    
    historical_sequences.append(display_sequence)
 
def generate_noise_term():
  global SIGMA, noise_term
 
  if IS_REPLAY and attempt_number <= len(historical_noise_terms):
    noise_term = historical_noise_terms[attempt_number - 1]
  else:
    noise_term = min(3, int(random.normalvariate(0, SIGMA)))
    noise_term = max(-3, noise_term)
 
    historical_noise_terms.append(noise_term)
 
def list_to_str(l):
  return ", ".join([
    str(elem)
    for elem in l
  ])
 
def round_to_int(num):
  if not "." in str(num):
    return int(num)
 
  dec = 0
  num = str(num)[:str(num).index('.')+dec+2]
  if num[-1]>='5':
    return int(float(num[:-2-(not dec)]+str(int(num[-2-(not dec)])+1)))
  return int(float(num[:-1]))
 
def bound_range_multiplier(num):
  global MIN_MULTIPLIER, MAX_MULTIPLIER
 
  ret = max(MIN_MULTIPLIER, num)
  ret = min(ret, MAX_MULTIPLIER)
 
  return ret
 
## Setup: Game history
historical_sequences = []
historical_noise_terms = []
historical_user_guesses = []
historical_ai_guesses = []
 
## Setup: AI
def init_ai_multipliers(random_guess=False):
  global n, ai_multiplier_guesses, MIN_MULTIPLIER, MAX_MULTIPLIER
 
  if random_guess:
    ai_multiplier_guesses = [
      random.randint(MIN_MULTIPLIER, MAX_MULTIPLIER)
      for i in range(n)
    ]
  else:
    ai_multiplier_guesses = [
      round_to_int(0.5 * (MIN_MULTIPLIER + MAX_MULTIPLIER))
      for i in range(n)
    ]
 
  return ai_multiplier_guesses
 
def update_ai_multipliers(sequence, passcode):
  global ai_multiplier_guesses, n
 
  if USE_STUDENT_AI:
    new_multipliers = calculate_new_multipliers(ai_multiplier_guesses, sequence, passcode, n)
  else:
    new_multipliers = calculate_new_multipliers_default(ai_multiplier_guesses, sequence, passcode, n)

  new_multipliers = [
    bound_range_multiplier(m)
    for m in new_multipliers    
  ]
 
  ai_multiplier_guesses = new_multipliers
 
  return ai_multiplier_guesses
 
def calculate_new_multipliers_default(current_multipliers, sequence, passcode, n):
  # Compute error term (y - h(x))
  products = [
    current_multipliers[i] * sequence[i]
    for i in range(n)
  ]
  passcode_guess = sum(products)
  error = passcode - passcode_guess
 
  # Set learning rate (alpha) according to "gap closing" heuristic
  learning_rate = 1. / sum([num*num for num in sequence])
 
  # Backpropagation: SGD update rule
  new_multipliers = [
    current_multipliers[i] + learning_rate * error * sequence[i]
    for i in range(n)
  ]
  
  return new_multipliers
 
## Setup: Layouts
difficulty_input = widgets.BoundedIntText(value=1, min=MIN_N, max=MAX_N)
noise_input = widgets.BoundedIntText(value=0, min=0, max=1)
difficulty_submit = widgets.Button(description="Submit", tooltip="Select difficulty")
difficulty_section = widgets.VBox([
  widgets.HTML('<h2>Select difficulty</h2>'),
  widgets.HTML('<h3>Set sequence length:</h3>'),
  difficulty_input,
  widgets.HTML('<h3>Enable noise (0=no, 1=yes):</h3>'),
  noise_input,
  difficulty_submit,
])
 
global lock_attempt_title
display_sequence_row = widgets.HBox([
  widgets.HTML('<strong>Hint Sequence:</strong>'),
  widgets.HTML('<strong>?</strong>'),
])
global guess_table
header_texts = ["Input Digit (1-9)", "Hint Sequence", "Product"]
guess_table_columns = [
  widgets.HTML(
    f"<strong>{header_texts[i]}</strong>",
    layout=widgets.Layout(width="auto", grid_area=f"col{i+1}")
  )
  for i in range(3)
]
global guess_table_rows
global multiplier_cells
global sequence_cells
global product_cells
separator_row = widgets.HTML('<hr></hr>')
team_guess_row = widgets.HBox([
  widgets.HTML('<strong>Passcode Guess:</strong>'),
  widgets.HTML('<strong>?</strong>'), 
])
passcode_row = widgets.HBox([
  widgets.HTML('<strong>Actual Passcode:</strong>'),
  widgets.HTML('<strong>?</strong>'),
])
lock_attempt_submit = widgets.Button(description="Submit Guess", tooltip="Submit guess")
lock_next_sequence = widgets.Button(description="Next Sequence", tooltip="Next sequence")
lock_use_ai_guess = widgets.Button(description="Use AI Guess", tooltip="Use AI guess")
message_row = widgets.HBox([
  widgets.HTML('<strong></strong>'), # Tally marker
  widgets.HTML('<strong></strong>'), # Correct or Incorrect
  widgets.HTML('<span></span>'), # Message
])
global lock_attempt_section
 
## Core game event handlers
def confirm_difficulty(button):
  global difficulty_input, noise_input, n, SIGMA, guess_leeway
  global difficulty_section
 
  # Finish setup requiring n
  n = difficulty_input.value
  SIGMA = noise_input.value

  if SIGMA == 0:
    guess_leeway = 0
  else:
    guess_leeway = 3

  generate_multipliers()
  init_ai_multipliers()
 
  display_lock_attempt_section()
  
  difficulty_section.layout.display = 'none'
 
def display_lock_attempt_section():
  global n
  global lock_attempt_title
  global display_sequence_row
  global guess_table
  global separator_row
  global team_guess_row
  global passcode_row
  global lock_attempt_submit
  global lock_next_sequence
  global message_row
  global lock_attempt_section
 
  global ALLOW_AI
 
  lock_attempt_title = widgets.HTML(attempt_header_html())
 
  init_guess_table()
 
  lock_buttons = [lock_attempt_submit, lock_next_sequence]
  if ALLOW_AI:
    lock_buttons.append(lock_use_ai_guess)
 
  lock_attempt_section = widgets.VBox([
    lock_attempt_title,
    display_sequence_row,
    guess_table,
    separator_row,
    team_guess_row,
    passcode_row,
    widgets.HBox(lock_buttons),
    message_row,
  ])
  display(lock_attempt_section)
 
  next_sequence(first_call=True)
 
def init_guess_table():
  global n, player_names, MIN_MULTIPLIER, MAX_MULTIPLIER
  global guess_table
  global guess_table_columns
  global guess_table_rows
  global multiplier_cells
  global sequence_cells
  global product_cells
 
  guess_table_rows = [
    widgets.HTML(
      f"<strong>Position {i+1}</strong>",
      layout=widgets.Layout(width="auto", grid_area=f"row{i+1}")
    )
    for i in range(n)
  ]
 
  multiplier_cells = [
    widgets.BoundedIntText(
      value=1,
      min=MIN_MULTIPLIER,
      max=MAX_MULTIPLIER,
      layout=widgets.Layout(width="auto", grid_area=f"cell{i+1}1")
    )
    for i in range(n)
  ]
 
  for cell in multiplier_cells:
    cell.observe(confirm_guesses, names='value')
 
  sequence_cells = [
    widgets.HTML(
      cell_text("?"),
      layout=widgets.Layout(width="auto", grid_area=f"cell{i+1}2")
    )
    for i in range(n)
  ]
 
  product_cells = [
    widgets.HTML(
      cell_text("?"),
      layout=widgets.Layout(width="auto", grid_area=f"cell{i+1}3")
    )
    for i in range(n)
  ]
 
  all_cells = []
  all_cells.extend(guess_table_columns)
  all_cells.extend(guess_table_rows)
  all_cells.extend(multiplier_cells)
  all_cells.extend(sequence_cells)
  all_cells.extend(product_cells)
 
  grid_template_areas = ['". col1 col2 col3"']
  for i in range(n):
    grid_template_areas.append(f'"row{i+1} cell{i+1}1 cell{i+1}2 cell{i+1}3"')
  
  grid_template_areas = "\n" + "\n".join(grid_template_areas) + "\n"
 
  guess_table = widgets.GridBox(
    children=all_cells,
    layout=widgets.Layout(
      width="100%",
      max_width="600px",
      grid_template_rows=" ".join(["auto" for i in range(n+1)]),
      grid_template_columns="25% 25% 25% 25%",
      grid_template_areas=grid_template_areas,
      grid_gap='5px 10px'
    )
  )
 
def next_sequence(first_call=False):
  global attempt_number, guessing_enabled, display_sequence
  global lock_attempt_title, sequence_cells, message_row, display_sequence_row
 
  # Update game flow state and update state UI
  set_enable_guessing(True)
  attempt_number += 1
  lock_attempt_title.value = attempt_header_html()
 
  clear_correct_message()
 
  # Generate sequence and update sequence UI
  generate_sequence()
  generate_noise_term()
 
  for i in range(n):
    sequence_cells[i].value = cell_text(display_sequence[i])
 
  display_sequence_row.children[1].value = cell_text(list_to_str(display_sequence))
 
  if not first_call:
    confirm_guesses(None)
  
  passcode_row.children[1].value = cell_text("?")
 
def confirm_guesses(change):
  global multiplier_guesses, display_sequence
  global multiplier_cells, product_cells, team_guess_row
 
  multiplier_guesses = [
    cell.value
    for cell in multiplier_cells
  ]
 
  products = [
    multiplier_guesses[i] * display_sequence[i]
    for i in range(n)
  ]
  team_guess = sum(products)
 
  for i in range(n):
    product_cells[i].value = cell_text(products[i])
 
  team_guess_row.children[1].value = cell_text(team_guess)
 
  return team_guess
 
def submit_guesses(button):
  global multipliers, noise_term, display_sequence, correct_tally, guess_leeway
  global passcode_row, multiplier_cells, ai_multiplier_guesses
 
  team_guess = confirm_guesses(None)
 
  # Show passcode
  passcode = noise_term + sum([
    multipliers[i] * display_sequence[i]
    for i in range(n)
  ])
 
  passcode_row.children[1].value = cell_text(passcode)
 
  error = team_guess - passcode
 
  if abs(error) <= guess_leeway:
    correct_tally += 1
    display_correct_message(True, error, correct_tally)
  else:
    correct_tally = 0
    display_correct_message(False, error, correct_tally)
  
  display_tally(correct_tally)
 
  # Update game state
  set_enable_guessing(False)
 
  # Track game history
  if not IS_REPLAY and not ALLOW_AI:
    historical_user_guesses.append([
      cell.value
      for cell in multiplier_cells
    ])
  
  if ALLOW_AI and IS_REPLAY:
    historical_ai_guesses.append([
      round_to_int(ai_multiplier_guesses[i])
      for i in range(n)
    ])

  # AI: Perform backpropagation
  update_ai_multipliers(display_sequence, passcode)
 
def fill_ai_guesses():
  global ai_multiplier_guesses, n
  global multiplier_cells
 
  for i in range(n):
    multiplier_cells[i].value = round_to_int(ai_multiplier_guesses[i])
 
def display_tally(correct_tally):
  global message_row
 
  check_str = "&#10004;" * correct_tally
 
  message_row.children[0].value = cell_text(check_str, color='green')
 
def set_enable_guessing(should_enable):
  global guessing_enabled
  global lock_attempt_submit, lock_next_sequence
 
  guessing_enabled = should_enable
 
  lock_attempt_submit.disabled = not guessing_enabled
  lock_next_sequence.disabled = guessing_enabled
 
def display_correct_message(is_correct, error, correct_tally):
  global message_row
 
  if not is_correct:
    direction_text = "high" if error > 0 else "low"
    message_row.children[1].value = cell_text("Incorrect.", bold=True, color='red')
    message_row.children[2].value = cell_text(
      f"Our guess was too {direction_text} ({str(error)}). Try updating the multipliers."
    )
  else:
    if correct_tally >= 3:
      message_row.children[1].value = cell_text("You win!", bold=True, color='green')
    else:
      message_row.children[1].value = cell_text("Correct!", bold=True, color='green')
 
def clear_correct_message():
  global message_row
 
  message_row.children[1].value = cell_text("", bold=True)
  message_row.children[2].value = cell_text("")
 
## Link handlers to UI events
difficulty_submit.on_click(confirm_difficulty)
lock_attempt_submit.on_click(submit_guesses)
lock_next_sequence.on_click(lambda b: next_sequence())
lock_use_ai_guess.on_click(lambda b: fill_ai_guesses())
 
# Initialize game loop
attempt_number = 0
correct_tally = 0
display(difficulty_section)

**Done playing? Run the cell below to see what the actual multipliers were.**

**To play again, re-run the cell above. Try increasing the difficulty!**

In [ ]:
## Reveal multipliers
print(f"The multipliers you guessed were: {list_to_str(multiplier_guesses)}")
print(f"The actual lock multipliers were: {list_to_str(multipliers)}")

## Discussion Questions
1. What parts of the game were easy? What parts were difficult?
2. How did you notice yourself learning as the game went on?
3. What are some strategies one can use for guessing the multipliers? What information helped you decide your new guesses?
4. In trying to guess the real multipliers, did seeing new sequences help us or hurt us?
5. [Bonus] If we were to guess the multipliers randomly each round, how many attempts would you expect that to take (roughly)? How does that compare to how many attempts we took?



## Building an AI to Play Escape Room

Believe it or not, this escape room game has deep connections to the foundations of artificial intelligence. A computer can "learn" to play this game using an algorithm called **backpropagation**. What's special is that this same algorithm is used by computers to perform "intelligent" tasks such as predicting housing prices or recognizing steal signs in baseball.

Before we dig into how backpropagation works, let's see how a computer player using backpropagation performed on our last game:

**Run the cell below to replay the previous game sequence using AI.**

**Click "Use AI Guess" to update your guesses before submitting.**

In [ ]:
## Replay AI backpropagation based on previous game state
ALLOW_AI = True
IS_REPLAY = True
init_ai_multipliers()

correct_tally = 0
attempt_number = 0

display_lock_attempt_section()
display_tally(correct_tally)
team_guess_row.children[1].value = cell_text("?")

How did the AI compare to our team's performance? Here's a graph comparing relative error (i.e. how close our guesses were to the real passcodes) over time:

In [ ]:
## Graph team error vs. AI error

## Graph relative performance & learning over time
m = len(historical_sequences)
n = len(multipliers)

user_abs_error = []
ai_abs_error = []

actual_totals = []
user_totals = []
ai_totals = []

user_multiplier_distance = []
ai_multiplier_distance = []

for j in range(m):
  sequence = historical_sequences[j]
  noise_term = historical_noise_terms[j]

  passcode = noise_term + sum([
    multipliers[i] * sequence[i]
    for i in range(n)
  ])

  actual_totals.append(passcode)

  if j < len(historical_user_guesses):
    user_total = sum([
      historical_user_guesses[j][i] * sequence[i]
      for i in range(n)
    ])
    user_totals.append(user_total)

    user_abs_error.append(abs(passcode - user_total))

    user_multiplier_distance.append(sum([
      abs(multipliers[i] - historical_user_guesses[j][i])
      for i in range(n)
    ]))

  if j < len(historical_ai_guesses):
    ai_total = sum([
      historical_ai_guesses[j][i] * sequence[i]
      for i in range(n)
    ])
    ai_totals.append(ai_total)

    ai_abs_error.append(abs(passcode - ai_total))

    ai_multiplier_distance.append(sum([
      abs(multipliers[i] - historical_ai_guesses[j][i])
      for i in range(n)
    ]))

figure, axes = pyplot.subplots()

user_error_line, = axes.plot([], [], 'b', label='Our team', linewidth=2)
ai_error_line, = axes.plot([], [], 'r', label='AI', linewidth=2)

min_error = min(min(user_abs_error), min(ai_abs_error))
max_error = max(max(user_abs_error), max(ai_abs_error))

axes.set_xlim((1, m))
axes.set_ylim((min_error,  max_error + 1))

axes.set_title("Passcode Error Over Time")
axes.set_xlabel('Attempt')
axes.set_ylabel('Passcode Error')

# Define animation
def init_animation():
  user_error_line.set_data([], [])
  ai_error_line.set_data([], [])

  return user_error_line, ai_error_line,

def animate(i):
  user_error_x = [j + 1 for j in range(min(i + 1, len(user_abs_error)))]
  user_error_y = user_abs_error[:i+1]

  ai_error_x = [j + 1 for j in range(min(i + 1, len(ai_abs_error)))]
  ai_error_y = ai_abs_error[:i+1]

  user_error_line.set_data(user_error_x, user_error_y)
  ai_error_line.set_data(ai_error_x, ai_error_y)

  return user_error_line, ai_error_line,

error_animation = animation.FuncAnimation(
    figure,
    animate,
    init_func=init_animation,
    frames=m+3,
    interval=300, 
    blit=True)

# Display UI
pyplot.legend()
pyplot.close()

HTML(error_animation.to_jshtml())

What do you notice about how accurately we were able to predict passcodes as the game progressed? How about the AI?

A similar graph shows how close we were to guessing the true multipliers:

In [ ]:
## Graph team distance vs. AI distance from multipliers
figure_dist, axes_dist = pyplot.subplots()

user_distance_line, = axes_dist.plot([], [], 'b', label='Our team', linewidth=2)
ai_distance_line, = axes_dist.plot([], [], 'r', label='AI', linewidth=2)

min_distance = min(min(user_multiplier_distance), min(ai_multiplier_distance))
max_distance = max(max(user_multiplier_distance), max(ai_multiplier_distance))

axes_dist.set_xlim((1, m))
axes_dist.set_ylim((0,  max_distance + 1))

axes_dist.set_title("Distance From True Multipliers Over Time")
axes_dist.set_xlabel('Attempt')
axes_dist.set_ylabel('Distance')

# Define animation
def init_animation_dist():
  user_distance_line.set_data([], [])
  ai_distance_line.set_data([], [])

  return user_distance_line, ai_distance_line,

def animate_dist(i):
  user_distance_x = [j + 1 for j in range(min(i + 1, len(user_multiplier_distance)))]
  user_distance_y = user_multiplier_distance[:i+1]

  ai_distance_x = [j + 1 for j in range(min(i + 1, len(ai_multiplier_distance)))]
  ai_distance_y = ai_multiplier_distance[:i+1]

  user_distance_line.set_data(user_distance_x, user_distance_y)
  ai_distance_line.set_data(ai_distance_x, ai_distance_y)

  return user_distance_line, ai_distance_line,

distance_animation = animation.FuncAnimation(
    figure_dist,
    animate_dist,
    init_func=init_animation_dist,
    frames=m+3,
    interval=300, 
    blit=True)

# Display UI
pyplot.legend()
pyplot.close()

HTML(distance_animation.to_jshtml())

## Understanding Backpropagation

Backpropagation is not a complicated algorithm. In fact, it can be simply understood using many of the techniques you may have already learned from playing Escape Room: Predicting Passcodes. At its core, backpropagation is built on three strategies from the game:

1. Every round, each multiplier can be updated (by adding or subtracting) based on feedback

How much to update each multiplier is proportional to:

2. The difference between the real passcode and our guess (i.e. "error")

3. The size of the number at each position in the sequence (i.e. "input")

**Notes**

To understand (1), you may have noticed that our multipliers in each round tended to be close to the multipliers from the previous round. This makes sense because we expect to be getting closer to the real answer over time.

As an example of (2), suppose the passcode for a given round was 50. Would you be more likely to update your guess if our team score was 45, or if it was 10?

The intuition behind (3) is more subtle. One way it can be understood is through large differences. Suppose the input sequence for a given round is "1, 5, 0", and the difference between our guess and the actual passcode is 20. In which position would the multiplier most likely need to change?

**And that's it.** Every multiplier can be updated according to this simple formula that captures the three strategies above:

`New_Multiplier = Current_Multiplier + (Passcode − Guess) * Input * Learning_Rate`

You've learned one of the most famous algorithms in artificial intelligence!

## Challenge: Build Your Own AI

Using what we've learned above, can you build your own AI to play the game? 

You'll find a playground below that will allow you to write your own Python code to play the game. Try to fill in the function `calculate_new_multipliers` to perform backpropagation, and run the code to see your algorithm in action.

### Details

`calculate_new_multipliers` is called each time a passcode guess is submitted. It is given the following information from the attempt:
- `current_multipliers`: a List of the current multiplier estimates (setting each multiplier to `5` on the first attempt)
- `sequence`: a List of all numbers in the input lock sequence
- `actual_passcode`: an integer storing the actual passcode returned by the lock
- `n`: the length of every sequence

Currently, `calculate_new_multipliers` chooses a random number between 1-9 for each multiplier. Can you update it to solve the game more effectively?

**Hint 1**: Try printing out each of the parameters `current_multipliers`, `sequence`, and `actual_pascode` to see how they work.

**Hint 2**: As a starter task: how would you use `current_multipliers` and `sequence` to calculate the passcode guess from the last round?

**Hint 3**: There are many ways to choose `learning_rate` in the backpropagation equation (and many strategies involve changing `learning_rate` over time). Try experimenting!

**Run the cell below to get started. (If it fails, make sure to run the first cell in this notebook if you haven't already)**

In [ ]:
## NOTE: make sure to copy and save your code if you want to keep it!
## Reloading the notebook will not save your changes.

## Update this function to build your own AI
def calculate_new_multipliers(current_multipliers, sequence, actual_passcode, n):
  new_multipliers = []

  # Algorithm: guess random multipliers
  for i in range(n):
    random_multiplier = random.randint(1, 9)
    new_multipliers.append(random_multiplier)
  
  return new_multipliers

## Do not modify the code below
USE_STUDENT_AI = True
ALLOW_AI = True
IS_REPLAY = False

# Initialize game loop
attempt_number = 0
correct_tally = 0
display_tally(correct_tally)
team_guess_row.children[1].value = cell_text("?")

difficulty_section.layout.display = 'block'
display(difficulty_section)